<a href="https://colab.research.google.com/github/GGMmattos/Modelos-de-Deteccao-de-Linguagem-Ofensiva/blob/main/R2_TOLD_BR_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing needed libraries
!pip install -U scikit-learn
!pip install -U spacy
!python -m spacy download pt
!pip install wandb
from pandas.core.window.ewm import OnlineExponentialMovingWindow
import pandas as pd
import numpy as np
import seaborn as sns #Para visualização de gráficos
import matplotlib.pyplot as plt #Para visualização de gráficos
import plotly.express as px #Biblioteca para geração de gráfico dinámico
%matplotlib inline
from collections import Counter
import re, nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import logging
import numpy as np
import sklearn
import pickle
import spacy
from sklearn.model_selection import train_test_split
!pip install simpletransformers "transformers==4.30.2"
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
told_br = pd.read_csv("/content/drive/MyDrive/PIC /Dataset/told_br_binary.csv")

Tranformando a base em binária

In [ ]:
told_br["toxic"] = 0
told_br["toxic_1"] = 0
told_br["toxic_2"] = 0
told_br["toxic_3"] = 0
told_br["homophobia"] = 0
told_br["obscene"] = 0
told_br["insult"] = 0
told_br["racism"] = 0
told_br["misogyny"] = 0
told_br["xenophobia"] = 0
for i, row in told_br.iterrows():
    count = 0
    if 1 in list(row[["homophobia_1", "obscene_1", "insult_1", "racism_1", "misogyny_1", "xenophobia_1"]]):
        told_br.loc[i, "toxic_1"] = 1
        count += 1
    if 1 in list(row[["homophobia_2", "obscene_2", "insult_2", "racism_2", "misogyny_2", "xenophobia_2"]]):
        told_br.loc[i, "toxic_2"] = 1
        count += 1
    if 1 in list(row[["homophobia_3", "obscene_3", "insult_3", "racism_3", "misogyny_3", "xenophobia_3"]]):
        told_br.loc[i, "toxic_3"] = 1
        count += 1

    told_br.loc[i, "homophobia"] = np.sum(list(row[["homophobia_1", "homophobia_2", "homophobia_3"]]))

    told_br.loc[i, "obscene"] = np.sum(list(row[["obscene_1", "obscene_2", "obscene_3"]]))

    told_br.loc[i, "insult"] = np.sum(list(row[["insult_1", "insult_2", "insult_3"]]))

    told_br.loc[i, "racism"] = np.sum(list(row[["racism_1", "racism_2", "racism_3"]]))

    told_br.loc[i, "misogyny"] = np.sum(list(row[["misogyny_1", "misogyny_2", "misogyny_3"]]))

    told_br.loc[i, "xenophobia"] = np.sum(list(row[["xenophobia_1", "xenophobia_2", "xenophobia_3"]]))

    if count >= 1:
        told_br.loc[i, "toxic"] = 1

In [ ]:
data_told = told_br[["text", "toxic", "homophobia", "obscene", "insult", "racism", "misogyny", "xenophobia"]]

Treinamento e preparação dos dados

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data
#data_told = data_told.iloc[:,[0, 1]]
#train,eva = train_test_split(data_told,test_size = 0.2)


train_df = pd.read_csv('/content/drive/MyDrive/PIC /Teste Treinamento/Told_BR/X_train.csv')
y_train = pd.read_csv('/content/drive/MyDrive/PIC /Teste Treinamento/Told_BR/y_train.csv')
train_df['toxic'] =  y_train['toxic']
train_df = train_df.iloc[:,[1, 2]]

eva_df = pd.read_csv('/content/drive/MyDrive/PIC /Teste Treinamento/Told_BR/X_test.csv')
y_test = pd.read_csv('/content/drive/MyDrive/PIC /Teste Treinamento/Told_BR/y_test.csv')
eva_df['toxic'] =  y_test['toxic']
eva_df = eva_df.iloc[:,[1, 2]]

train_df = pd.DataFrame({
   'text': train_df['text'],
   'label': train_df['toxic']
})

eva_df = pd.DataFrame({
   'text': eva_df['text'],
   'label': eva_df['toxic']
})

#Optional model configuration
model_args = ClassificationArgs(
  num_train_epochs=1,
  overwrite_output_dir=True
                                )
#Create a ClassificationModel
model = ClassificationModel(
    "bert", "neuralmind/bert-base-portuguese-cased", args=model_args,
    use_cuda=False
)

model.train_model(train_df)

# Save model
with open('TOLD_BR-BERT.pkl ', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eva_df)
lst = []
for arr in model_outputs:
    lst.append(np.argmax(arr))
true = eva_df['label'].tolist()
predicted = lst
print(classification_report(true, predicted))

              precision    recall  f1-score   support

           0       0.85      0.73      0.79      2349
           1       0.71      0.84      0.77      1851

    accuracy                           0.78      4200
   macro avg       0.78      0.79      0.78      4200
weighted avg       0.79      0.78      0.78      4200



In [ ]:
confusion_matrix(true, predicted)

array([[1717,  632],
       [ 294, 1557]])

Função para pré-processamento

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# base_hatebr = pd.read_csv('/content/drive/MyDrive/PIC /Dataset/HateBR.csv')
# base_hatebr_train_data = base_hatebr.iloc[:,[0, 1]]
# train,eva = train_test_split(base_hatebr_train_data,test_size = 0.2)


train_df = pd.read_csv('/content/drive/MyDrive/PIC /Teste Treinamento/Told_BR/Pre processados/X_train_p.csv')
y_train = pd.read_csv('/content/drive/MyDrive/PIC /Teste Treinamento/Told_BR/y_train.csv')
train_df['toxic'] =  y_train['toxic']
train_df = train_df.iloc[:,[1, 2]]

eva_df = pd.read_csv('/content/drive/MyDrive/PIC /Teste Treinamento/Told_BR/Pre processados/X_test_p.csv')
y_test = pd.read_csv('/content/drive/MyDrive/PIC /Teste Treinamento/Told_BR/y_test.csv')
eva_df['toxic'] =  y_test['toxic']
eva_df = eva_df.iloc[:,[1, 2]]


train_df = pd.DataFrame({
     'text': train_df['text'],
     'label': train_df['toxic']
 })

eva_df = pd.DataFrame({
     'text': eva_df['text'],
     'label': eva_df['toxic']
 })

model_args = ClassificationArgs(num_train_epochs=1)

model = ClassificationModel(
    "bert", "neuralmind/bert-base-portuguese-cased", args=model_args,
    use_cuda=False
)



In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eva_df)
lst = []
for arr in model_outputs:
    lst.append(np.argmax(arr))
true = eva_df['label'].tolist()
predicted = lst
print(classification_report(true, predicted))

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/4200 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/525 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.83      0.73      0.78      2349
           1       0.71      0.81      0.76      1851

    accuracy                           0.77      4200
   macro avg       0.77      0.77      0.77      4200
weighted avg       0.78      0.77      0.77      4200

